<a href="https://colab.research.google.com/github/e754/Glucose-Research/blob/main/logisticRegressionGlucoseDayOne.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report
from sklearn.utils import resample 
from imblearn.over_sampling import SMOTE
import numpy as np

In [2]:
from sklearn.preprocessing import LabelEncoder
import pandas as pd
import os
current_path = os.path.abspath('.')
label_encoder = LabelEncoder()
fi=os.path.abspath('../../data/cohortedData.csv')
print(fi)
data = pd.read_csv(fi)


/Users/yugang/Documents/GitHub/Glucose-Research/data/cohortedData.csv


In [3]:
data['race'].unique()

array(['BLACK/AFRICAN AMERICAN', 'WHITE', 'BLACK/CAPE VERDEAN',
       'ASIAN - SOUTH EAST ASIAN', 'WHITE - BRAZILIAN',
       'HISPANIC OR LATINO', 'WHITE - OTHER EUROPEAN', 'BLACK/AFRICAN',
       'BLACK/CARIBBEAN ISLAND', 'ASIAN',
       'HISPANIC/LATINO - PUERTO RICAN', 'WHITE - RUSSIAN',
       'ASIAN - CHINESE', 'WHITE - EASTERN EUROPEAN',
       'HISPANIC/LATINO - DOMINICAN', 'ASIAN - ASIAN INDIAN',
       'HISPANIC/LATINO - HONDURAN', 'HISPANIC/LATINO - COLUMBIAN',
       'HISPANIC/LATINO - CENTRAL AMERICAN',
       'HISPANIC/LATINO - GUATEMALAN', 'HISPANIC/LATINO - MEXICAN',
       'ASIAN - KOREAN', 'HISPANIC/LATINO - CUBAN',
       'HISPANIC/LATINO - SALVADORAN'], dtype=object)

In [4]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 24927 entries, 0 to 24926
Data columns (total 51 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   subject_id                  24927 non-null  int64  
 1   hadm_id                     24927 non-null  int64  
 2   stay_id                     24927 non-null  int64  
 3   first_careunit              24927 non-null  object 
 4   last_careunit               24927 non-null  object 
 5   intime                      24927 non-null  object 
 6   outtime                     24927 non-null  object 
 7   los                         24927 non-null  float64
 8   race                        24927 non-null  object 
 9   language                    24927 non-null  object 
 10  admElective                 24927 non-null  bool   
 11  age                         24927 non-null  int64  
 12  gender                      24927 non-null  object 
 13  anchor_year_group           249

In [5]:
categorical_vars = ['anchor_year_group', 'race_group','diabetes_types']
dummy_vars = pd.get_dummies(data[categorical_vars])

In [6]:
data.fillna(0, inplace=True)
data['diabetes_types'] = data['diabetes_types'].replace(0, 'none')
data['diabetes_types'] = data['diabetes_types'].replace(1.0, 'one')
data['diabetes_types'] = data['diabetes_types'].replace(2.0, 'two')
data['race_group'] = data['race_group'].replace('OTHER', 'Other')

data['admElective'] = label_encoder.fit_transform(data['admElective'])
data['hadInsulinDayOne'] = label_encoder.fit_transform(data['hadInsulinDayOne'])
data['hadMeasurmentDayOne'] = label_encoder.fit_transform(data['hadMeasurmentDayOne'])
niceColumns=["uti", "biliary", "skin",'SOFA', 'hypertension_present','heart_failure_present','copd_present','cad_present']

categorical_vars = ['anchor_year_group', 'race_group','diabetes_types']
dummy_vars = pd.get_dummies(data[categorical_vars])
dummy_vars = dummy_vars.drop(columns=['race_group_White'])
dummy_vars = dummy_vars.drop(columns=['anchor_year_group_2008 - 2010'])
dummy_vars = dummy_vars.drop(columns=['diabetes_types_none'])

data = pd.concat([data, dummy_vars], axis=1)


continuous_vars = ['ckd_stages','age','charlson_comorbidity_index','diabetes_with_cc','diabetes_without_cc']
#,'sofa','cardiovascular','cns','coagulation','liver','renal','respiration'
independent = data[continuous_vars + list(dummy_vars.columns)+niceColumns]
dependent = data['hadMeasurmentDayOne']

In [7]:
!pip install matplotlib
from sklearn.metrics import roc_curve
import matplotlib.pyplot as plt
def test_model(X_test, y_test, model):

    y_pred = model.predict(X_test)
    print(classification_report(y_true=y_test, y_pred=y_pred))
    fpr, tpr, _ = roc_curve(y_test,  y_pred)

    #create ROC curve
    plt.plot(fpr,tpr)
    plt.title('ROC Curve')
    plt.ylabel('True Positive Rate')
    plt.xlabel('False Positive Rate')
    plt.show()

In [9]:
#all patients
import statsmodels.api as sm



X_train, X_test, y_train, y_test = train_test_split(independent, dependent, test_size=0.25, random_state=16)



minority_x = X_train[y_train==0]
minority_y = y_train[y_train==0]
majority_x = X_train[y_train==1]
majority_y = y_train[y_train==1]

# Perform oversampling on the minority class
minority_size = len(minority_x)
majority_size = len(majority_x)

oversampled_minority_x, oversampled_minority_y = resample(
minority_x, minority_y, replace=True, n_samples=len(majority_x)
)


# Combine oversampled minority and majority
x_train_final = pd.concat([oversampled_minority_x, majority_x])
y_train_final = pd.concat([oversampled_minority_y, majority_y])

logit_model = sm.Logit(y_train_final, x_train_final)
logit_model.fit_regularized(start_params=None,method = 'l1',alpha = 0)
result = logit_model.fit()
odds_ratio = np.exp(result.params)
ci = np.exp(result.conf_int())

# Create a DataFrame to display the results
results_df = pd.DataFrame({'Odds Ratio': odds_ratio, 'Lower CI': ci[0], 'Upper CI': ci[1]})
print(results_df)
# Find the odds ratio and its confidence interval


ValueError: Pandas data cast to numpy dtype of object. Check input data with np.asarray(data).

In [ ]:
y_test

754      0
16149    1
21886    1
26850    1
14257    1
        ..
17126    1
4481     1
7816     1
25651    1
28863    1
Name: hadMeasurmentDayOne, Length: 7402, dtype: int64

In [ ]:
predicted_probs = result.predict(X_test)

# Convert predicted probabilities to binary predictions (0 or 1)
predicted_values = (predicted_probs >= 0.5).astype(int)

# Compare predicted values with actual values and calculate accuracy
accuracy = (predicted_values == y_test).mean()


In [ ]:
accuracy

0.5210753850310726

In [ ]:
result.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            Logit Regression Results                           
===============================================================================
Dep. Variable:     hadMeasurmentDayOne   No. Observations:                40698
Model:                           Logit   Df Residuals:                    40676
Method:                            MLE   Df Model:                           21
Date:                 Fri, 07 Jul 2023   Pseudo R-squ.:                 0.03008
Time:                         22:58:47   Log-Likelihood:                -27361.
converged:                        True   LL-Null:                       -28210.
Covariance Type:             nonrobust   LLR p-value:                     0.000
=================================================================================================
                                    coef    std err          z      P>|z|      [0.025      0.975]
-------------------------------------------------------------------------------------------------
ckd_stages                       -0.0016      0.013     -0.118      0.906      -0.027       0.024
age                              -0.0068      0.001    -12.130      0.000      -0.008      -0.006
charlson_comorbidity_index        0.0217      0.005      4.704      0.000       0.013       0.031
diabetes_with_cc                  0.1503      0.042      3.595      0.000       0.068       0.232
diabetes_without_cc               0.5762      0.029     19.977      0.000       0.520       0.633
anchor_year_group_2011 - 2013    -0.0301      0.025     -1.190      0.234      -0.080       0.020
anchor_year_group_2014 - 2016     0.1607      0.028      5.834      0.000       0.107       0.215
anchor_year_group_2017 - 2019     0.1132      0.035      3.262      0.001       0.045       0.181
race_Asian                       -0.5078      0.061     -8.363      0.000      -0.627      -0.389
race_Black                       -0.3120      0.040     -7.831      0.000      -0.390      -0.234
race_Hispanic                    -0.1845      0.058     -3.165      0.002      -0.299      -0.070
race_White                       -0.1428      0.027     -5.344      0.000      -0.195      -0.090
diabetes_types_one                0.2580      0.116      2.232      0.026       0.031       0.485
diabetes_types_two                0.2984      0.052      5.771      0.000       0.197       0.400
uti                              -0.6471      0.121     -5.356      0.000      -0.884      -0.410
biliary                          -1.4249      0.166     -8.605      0.000      -1.749      -1.100
skin                             -1.1938      0.222     -5.371      0.000      -1.629      -0.758
sofa                              0.0529      0.003     18.130      0.000       0.047       0.059
hypertension_present             -0.0084      0.034     -0.249      0.803      -0.075       0.058
heart_failure_present             0.1638      0.040      4.129      0.000       0.086       0.242
copd_present                     -0.1287      0.040     -3.236      0.001      -0.207      -0.051
cad_present                      -0.0115      0.039     -0.298      0.766      -0.087       0.064
=================================================================================================
"""